<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/DataWarehousing_with_MySQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![CC-BY-SA](https://licensebuttons.net/l/by-sa/3.0/88x31.png)<br>
<hr>

![alt text](http://1.bp.blogspot.com/-nqAGzznZQNo/UwS8rxjfXeI/AAAAAAAABTA/nunmRLowpps/s1600/PraxisLogo.gif)<br>
[Data Science Program](http://praxis.ac.in/Programs/business-analytics/)

<hr>

[Prithwis Mukerjee](http://www.yantrajaal.com)<br>

#Datawarehousing with MySQL <br>

<br>
based on the codes available with the book Dimensional Datawarehousing with MySQL https://brainysoftware.com/9780975212820   [archive](https://drive.google.com/open?id=1BeIC5uEMGu85PADHRN-Zpn3lmZ3ihrkE)

# Install MySQL

In [ ]:
!apt -y install mysql-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libencode-locale-perl libevent-core-2.1-6
  libfcgi-perl libhtml-parser-perl libhtml-tagset-perl libhtml-template-perl
  libhttp-date-perl libhttp-message-perl libio-html-perl
  liblwp-mediatypes-perl libtimedate-perl liburi-perl mysql-client-5.7
  mysql-client-core-5.7 mysql-server-5.7 mysql-server-core-5.7
Suggested packages:
  libdata-dump-perl libipc-sharedcache-perl libwww-perl mailx tinyca
The following NEW packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libencode-locale-perl libevent-core-2.1-6
  libfcgi-perl libhtml-parser-perl libhtml-tagset-perl libhtml-template-perl
  libhttp-date-perl libhttp-message-perl libio-html-perl
  liblwp-mediatypes-perl libtimedate-perl liburi-perl mysql-client-5.7
  mysql-client-core-5.7 mysql-server mysql-server-5.7 mysql-server-core-5.7
0 upgraded, 19 

In [ ]:
!/etc/init.d/mysql restart

 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
No directory, logging in with HOME=/
   ...done.


In [ ]:
!mysql -e 'show databases'

+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| sys                |
+--------------------+


In [ ]:
!mysql -e 'create database praxisDB'

# Get Code Files<br>
 

In [ ]:
#!wget https://brainysoftware.com/source/9780975212820.zip
#https://drive.google.com/open?id=1qaiPXbxtIivCbBLmWiYvHMKAjAmNn2Y0
!gdown https://drive.google.com/uc?id=1qaiPXbxtIivCbBLmWiYvHMKAjAmNn2Y0

Downloading...
From: https://drive.google.com/uc?id=1qaiPXbxtIivCbBLmWiYvHMKAjAmNn2Y0
To: /content/9780975212820.zip
100% 166k/166k [00:00<00:00, 1.46MB/s]


In [ ]:
!unzip 9780975212820.zip

Archive:  9780975212820.zip
   creating: scripts/
  inflating: scripts/across_all_dimensions.sql  
  inflating: scripts/across_date.sql  
  inflating: scripts/across_date_customer.sql  
  inflating: scripts/across_date_product.sql  
  inflating: scripts/additive_data.sql  
  inflating: scripts/add_campaign_session.sql  
  inflating: scripts/add_four_milestones.sql  
  inflating: scripts/add_sales_order_quantity.sql  
  inflating: scripts/add_two_sales_orders.sql  
  inflating: scripts/annual_aggregation.sql  
  inflating: scripts/annual_tables.sql  
  inflating: scripts/balance_across_months.sql  
  inflating: scripts/balance_across_products.sql  
  inflating: scripts/band_dim.sql    
  inflating: scripts/band_ini.sql    
  inflating: scripts/band_regular.sql  
  inflating: scripts/campaign_session.sql  
  inflating: scripts/campaign_session_20.sql  
  inflating: scripts/campaign_session_21.sql  
  inflating: scripts/campaign_session_path.sql  
  inflating: scripts/campaign_stg_20.sql 

In [ ]:
!ls

9780975212820.zip  sample_data	scripts  source_data


# Initial Setup

In [ ]:
!ls scripts

 across_all_dimensions.sql		   loyal_customer.sql
 across_date_customer.sql		   month_end_balance_fact.sql
 across_date_product.sql		   month_end_balance_initial.sql
 across_date.sql			   month_end_balance_regular.sql
 add_campaign_session.sql		   month_end_sales_order_22.sql
 add_four_milestones.sql		   month_end_sales_order.sql
 additive_data.sql			   monthly_product_performer.sql
 add_sales_order_quantity.sql		   monthly_storage.sql
 add_two_sales_orders.sql		   month_rollup_dim.sql
 annual_aggregation.sql			   more_customer_sk.sql
 annual_tables.sql			   more_sales_order.sql
 balance_across_months.sql		   mysql_fix_privilege_tables.sql
 balance_across_products.sql		   new_customer_credit_check.sql
 band_dim.sql				   non_pa_customer.sql
 band_ini.sql				   nullify_campaign_session.sql
 band_regular.sql			   on_demand.sql
 campaign_session_20.sql		   order_quantity.sql
 campaign_session_21.sql		   order_status.sql
 campaign_session_path.sql		   pa_customer.sql
 campaign_session.sql	

In [ ]:
#Create userid
#
!mysql < scripts/create_user_id.sql

In [ ]:
#see the script that created the userid
#
!cat scripts/create_user_id.sql

/*********************************************************************/
/*                                                                   */
/* create_user_id.sql                                                */
/*                                                                   */
/*********************************************************************/

GRANT ALL ON *.* TO dwid@localhost IDENTIFIED BY 'pw'
;

/* end of script                                                     */


In [ ]:
# see the privileges given to dw
#
!mysql -e "show grants for dwid@localhost"

+---------------------------------------------------+
| Grants for dwid@localhost                         |
+---------------------------------------------------+
| GRANT ALL PRIVILEGES ON *.* TO 'dwid'@'localhost' |
+---------------------------------------------------+


In [ ]:
#Create databases
#
!mysql -udwid -ppw < scripts/create_databases.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
# see what the script had
#
!cat scripts/create_databases.sql

/*********************************************************************/
/*                                                                   */
/* create_databases.sql                                              */                  
/*                                                                   */
/*********************************************************************/

CREATE DATABASE dw
;

CREATE DATABASE source
;

/* end of script                                                     */


In [ ]:
!mysql -e "show databases"

+--------------------+
| Database           |
+--------------------+
| information_schema |
| dw                 |
| mysql              |
| performance_schema |
| praxisDB           |
| source             |
| sys                |
+--------------------+


In [ ]:
#create the various tables
# one fact table and three dimension tables
#
!mysql -udwid -ppw < scripts/create_dw_tables.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!cat scripts/create_dw_tables.sql

/*********************************************************************/
/*                                                                   */
/* create_dw_tables.sql                                              */
/*                                                                   */
/*********************************************************************/

/* default to dw database                                            */

USE dw;

/* creating customer_dim table                                       */

CREATE TABLE customer_dim 
( customer_sk INT NOT NULL AUTO_INCREMENT PRIMARY KEY
, customer_number INT
, customer_name CHAR(50)
, customer_street_address CHAR(50)
, customer_zip_code INT(5)
, customer_city CHAR(30)
, customer_state CHAR(2)
, effective_date DATE
, expiry_date DATE )
;

/* creating product_dim table                                                */

CREATE TABLE product_dim 
( product_sk INT NOT NULL AUTO_INCREMENT PRIMARY KEY
, product_code INT
, product_name CHAR

In [ ]:
!mysql dw -e "show tables"

+------------------+
| Tables_in_dw     |
+------------------+
| customer_dim     |
| date_dim         |
| order_dim        |
| product_dim      |
| sales_order_fact |
+------------------+


In [ ]:
!mysql dw -e "desc customer_dim"

+-------------------------+----------+------+-----+---------+----------------+
| Field                   | Type     | Null | Key | Default | Extra          |
+-------------------------+----------+------+-----+---------+----------------+
| customer_sk             | int(11)  | NO   | PRI | NULL    | auto_increment |
| customer_number         | int(11)  | YES  |     | NULL    |                |
| customer_name           | char(50) | YES  |     | NULL    |                |
| customer_street_address | char(50) | YES  |     | NULL    |                |
| customer_zip_code       | int(5)   | YES  |     | NULL    |                |
| customer_city           | char(30) | YES  |     | NULL    |                |
| customer_state          | char(2)  | YES  |     | NULL    |                |
| effective_date          | date     | YES  |     | NULL    |                |
| expiry_date             | date     | YES  |     | NULL    |                |
+-------------------------+----------+------+-----+-

In [ ]:
!mysql dw -e "desc product_dim"

+------------------+----------+------+-----+---------+----------------+
| Field            | Type     | Null | Key | Default | Extra          |
+------------------+----------+------+-----+---------+----------------+
| product_sk       | int(11)  | NO   | PRI | NULL    | auto_increment |
| product_code     | int(11)  | YES  |     | NULL    |                |
| product_name     | char(30) | YES  |     | NULL    |                |
| product_category | char(30) | YES  |     | NULL    |                |
| effective_date   | date     | YES  |     | NULL    |                |
| expiry_date      | date     | YES  |     | NULL    |                |
+------------------+----------+------+-----+---------+----------------+


In [ ]:
!mysql dw -e "desc order_dim"

+----------------+---------+------+-----+---------+----------------+
| Field          | Type    | Null | Key | Default | Extra          |
+----------------+---------+------+-----+---------+----------------+
| order_sk       | int(11) | NO   | PRI | NULL    | auto_increment |
| order_number   | int(11) | YES  |     | NULL    |                |
| effective_date | date    | YES  |     | NULL    |                |
| expiry_date    | date    | YES  |     | NULL    |                |
+----------------+---------+------+-----+---------+----------------+


In [ ]:
!date

Tue Apr  6 11:39:26 UTC 2021


In [ ]:
#this action cannot be performed except on a local machine 
!sudo date +%Y%m%d -s "20070201"

date: cannot set date: Operation not permitted
20070201


In [ ]:
!mysql -udwid -ppw < scripts/customer_sk.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql -udwid -ppw < scripts/more_customer_sk.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "select * from customer_dim"
# note we could not set the date to an earlier date by changing the computer time

+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
| customer_sk | customer_number | customer_name    | customer_street_address | customer_zip_code | customer_city | customer_state | effective_date | expiry_date |
+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
|           1 |               1 | Big Customers    | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             | 2021-04-06     | 9999-12-31  |
|           2 |               2 | Small Stores     | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             | 2021-04-06     | 9999-12-31  |
|           3 |               3 | Medium Retailers | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             | 2021-04-06     | 9999-12-31  |
|           4 |       

In [ ]:
#since it was not possible to set the system date on Colab, the date was changed manually
!mysql dw -e "update customer_dim set effective_date = '2007-02-01' "

In [ ]:
!mysql dw -e "select * from customer_dim"

+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
| customer_sk | customer_number | customer_name    | customer_street_address | customer_zip_code | customer_city | customer_state | effective_date | expiry_date |
+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
|           1 |               1 | Big Customers    | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             | 2007-02-01     | 9999-12-31  |
|           2 |               2 | Small Stores     | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             | 2007-02-01     | 9999-12-31  |
|           3 |               3 | Medium Retailers | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             | 2007-02-01     | 9999-12-31  |
|           4 |       

# Chap 2 - Slowly Changing Dimensions

**SCD1**


---







In [ ]:
!ls source_data

 campaign_session.csv   factory23.csv		    promo_schedule.csv
 customer10.csv         non_straight_campaign.csv   ragged_campaign.csv
 customer23.csv         product2-1.txt		   'readme 1st for data.txt'
 customer2.csv	        product21.txt		    zip_code.csv
 customer8.csv	        product2-2.txt
 factory19.csv	        product8.txt


In [ ]:
!cat source_data/customer2.csv

CUSTOMER NO,CUSTOMER NAME,STREET ADDRESS,ZIP CODE,CITY,STATE
1,Really Large Customers,7500 Louise Dr.,17050,Mechanicsburg,PA
2,Small Stores,2500 Woodland St.,17055,Pittsburgh,PA
3,Medium Retailers,1111 Ritter Rd.,17055,Pittsburgh,PA
4,Good Companies,9500 Scott St.,17050,Mechanicsburg,PA
5,Wonderful Shops,3333 Rossmoyne Rd.,17050,Mechanicsburg,PA
6,Loyal Clients,7070 Ritter Rd.,17055,Pittsburgh,PA
7,Distinguished Partners,9999 Scott St.,17050,Mechanicsburg,PA

the downloaded script cannot be used because the csv file name and directory is different from what is there in the script. hence two additional script files are dynamically created and used


---



In [ ]:
%%writefile pm_create_customer_stg.sql
/* default database to dw */
USE dw;
/* create customer_stg table */
CREATE TABLE customer_stg
(customer_number INT,
customer_name CHAR (30),
customer_street_address CHAR (30),
customer_zip_code INT (5),
customer_city CHAR (30),
customer_state CHAR (2) )
;
/* clean up customer_stg table and load customer.csv */
TRUNCATE customer_stg;

Writing pm_create_customer_stg.sql


In [ ]:
!mysql -udwid -ppw --table < pm_create_customer_stg.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "desc customer_stg"

+-------------------------+----------+------+-----+---------+-------+
| Field                   | Type     | Null | Key | Default | Extra |
+-------------------------+----------+------+-----+---------+-------+
| customer_number         | int(11)  | YES  |     | NULL    |       |
| customer_name           | char(30) | YES  |     | NULL    |       |
| customer_street_address | char(30) | YES  |     | NULL    |       |
| customer_zip_code       | int(5)   | YES  |     | NULL    |       |
| customer_city           | char(30) | YES  |     | NULL    |       |
| customer_state          | char(2)  | YES  |     | NULL    |       |
+-------------------------+----------+------+-----+---------+-------+


In [ ]:
%%writefile pm_load_customer_stg.sql
/* default database to dw */
USE dw;
LOAD DATA LOCAL INFILE 'source_data/customer2.csv'
INTO TABLE customer_stg
FIELDS TERMINATED BY ','
OPTIONALLY ENCLOSED BY ""
LINES TERMINATED BY '\r\n'
IGNORE 1 LINES
( customer_number
, customer_name
, customer_street_address
, customer_zip_code
, customer_city
, customer_state )
;
/* end of script */

Writing pm_load_customer_stg.sql


In [ ]:
!mysql -udwid -ppw --table < pm_load_customer_stg.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "select * from customer_stg"

+-----------------+------------------------+-------------------------+-------------------+---------------+----------------+
| customer_number | customer_name          | customer_street_address | customer_zip_code | customer_city | customer_state |
+-----------------+------------------------+-------------------------+-------------------+---------------+----------------+
|               1 | Really Large Customers | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             |
|               2 | Small Stores           | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             |
|               3 | Medium Retailers       | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             |
|               4 | Good Companies         | 9500 Scott St.          |             17050 | Mechanicsburg | PA             |
|               5 | Wonderful Shops        | 3333 Rossmoyne Rd.      |             17050 | Mechanicsburg | PA             |
|       

In [ ]:
!mysql -udwid -ppw < scripts/scd1.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "select * from customer_dim"

+-------------+-----------------+------------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
| customer_sk | customer_number | customer_name          | customer_street_address | customer_zip_code | customer_city | customer_state | effective_date | expiry_date |
+-------------+-----------------+------------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
|           1 |               1 | Really Large Customers | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             | 2007-02-01     | 9999-12-31  |
|           2 |               2 | Small Stores           | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             | 2007-02-01     | 9999-12-31  |
|           3 |               3 | Medium Retailers       | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             | 2007-02-01     | 

In [ ]:
#since it was not possible to set the system date on Colab, the date was changed manually
!mysql dw -e "update customer_dim set effective_date = '2007-02-02' where customer_number = 7 and customer_name = 'Distinguished Partners' "

In [ ]:
!mysql dw -e "select * from customer_dim"

+-------------+-----------------+------------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
| customer_sk | customer_number | customer_name          | customer_street_address | customer_zip_code | customer_city | customer_state | effective_date | expiry_date |
+-------------+-----------------+------------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
|           1 |               1 | Really Large Customers | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             | 2007-02-01     | 9999-12-31  |
|           2 |               2 | Small Stores           | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             | 2007-02-01     | 9999-12-31  |
|           3 |               3 | Medium Retailers       | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             | 2007-02-01     | 

**SCD2**


---



In [ ]:
!ls

9780975212820.zip	    pm_load_customer_stg.sql  scripts
pm_create_customer_stg.sql  sample_data		      source_data


In [ ]:
!ls scripts

 across_all_dimensions.sql		   loyal_customer.sql
 across_date_customer.sql		   month_end_balance_fact.sql
 across_date_product.sql		   month_end_balance_initial.sql
 across_date.sql			   month_end_balance_regular.sql
 add_campaign_session.sql		   month_end_sales_order_22.sql
 add_four_milestones.sql		   month_end_sales_order.sql
 additive_data.sql			   monthly_product_performer.sql
 add_sales_order_quantity.sql		   monthly_storage.sql
 add_two_sales_orders.sql		   month_rollup_dim.sql
 annual_aggregation.sql			   more_customer_sk.sql
 annual_tables.sql			   more_sales_order.sql
 balance_across_months.sql		   mysql_fix_privilege_tables.sql
 balance_across_products.sql		   new_customer_credit_check.sql
 band_dim.sql				   non_pa_customer.sql
 band_ini.sql				   nullify_campaign_session.sql
 band_regular.sql			   on_demand.sql
 campaign_session_20.sql		   order_quantity.sql
 campaign_session_21.sql		   order_status.sql
 campaign_session_path.sql		   pa_customer.sql
 campaign_session.sql	

In [ ]:
!mysql -udwid -ppw dw -e "truncate product_dim"

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "select * from product_dim"

In [ ]:
!cat source_data/product2-1.txt

PRODUCT CODE,PRODUCT NAME,PRODUCT GROUP
1          Hard Disk                     Storage
2          Floppy Drive                  Storage

In [ ]:
!mysql -udwid -ppw --table < scripts/create_product_stg.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "desc product_stg"

+------------------+----------+------+-----+---------+-------+
| Field            | Type     | Null | Key | Default | Extra |
+------------------+----------+------+-----+---------+-------+
| product_code     | int(11)  | YES  |     | NULL    |       |
| product_name     | char(30) | YES  |     | NULL    |       |
| product_category | char(30) | YES  |     | NULL    |       |
+------------------+----------+------+-----+---------+-------+


In [ ]:
%%writefile pm_load_prod_stg_01.sql
/* default database to dw */
USE dw;
/* clean up the staging table */
TRUNCATE product_stg;
/* use LOAD DATA INFILE */
LOAD DATA local INFILE 'source_data/product2-1.txt'
INTO TABLE product_stg
FIELDS TERMINATED BY ''
OPTIONALLY ENCLOSED BY ''
LINES TERMINATED BY '\r\n'
IGNORE 1 LINES
( product_code
, product_name
, product_category )
;
/* end of script */

Writing pm_load_prod_stg_01.sql


In [ ]:
!mysql -udwid -ppw --table < pm_load_prod_stg_01.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "select * from product_stg"

+--------------+--------------+------------------+
| product_code | product_name | product_category |
+--------------+--------------+------------------+
|            1 | Hard Disk    | Storage          |
|            2 | Floppy Drive | Storage          |
+--------------+--------------+------------------+


In [ ]:
!mysql -udwid -ppw --table < scripts/scd2.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "select * from product_dim"

+------------+--------------+--------------+------------------+----------------+-------------+
| product_sk | product_code | product_name | product_category | effective_date | expiry_date |
+------------+--------------+--------------+------------------+----------------+-------------+
|          1 |            1 | Hard Disk    | Storage          | 2021-04-06     | 9999-12-31  |
|          2 |            2 | Floppy Drive | Storage          | 2021-04-06     | 9999-12-31  |
+------------+--------------+--------------+------------------+----------------+-------------+


In [ ]:
!mysql dw -e "update product_dim set effective_date = '2007-02-03'"

In [ ]:
!mysql dw -e "select * from product_dim"

+------------+--------------+--------------+------------------+----------------+-------------+
| product_sk | product_code | product_name | product_category | effective_date | expiry_date |
+------------+--------------+--------------+------------------+----------------+-------------+
|          1 |            1 | Hard Disk    | Storage          | 2007-02-03     | 9999-12-31  |
|          2 |            2 | Floppy Drive | Storage          | 2007-02-03     | 9999-12-31  |
+------------+--------------+--------------+------------------+----------------+-------------+


In [ ]:
%%writefile pm_load_prod_stg_02.sql
/* default database to dw */
USE dw;
/* clean up the staging table */
TRUNCATE product_stg;
/* use LOAD DATA INFILE */
LOAD DATA local INFILE 'source_data/product2-2.txt'
INTO TABLE product_stg
FIELDS TERMINATED BY ''
OPTIONALLY ENCLOSED BY ''
LINES TERMINATED BY '\r\n'
IGNORE 1 LINES
( product_code
, product_name
, product_category )
;
/* end of script */

Writing pm_load_prod_stg_02.sql


In [ ]:
!mysql -udwid -ppw --table < pm_load_prod_stg_02.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "select * from product_stg"

+--------------+-----------------+------------------+
| product_code | product_name    | product_category |
+--------------+-----------------+------------------+
|            1 | Hard Disk Drive | Storage          |
|            2 | Floppy Drive    | Storage          |
|            3 | LCD Panel       | Monitor          |
+--------------+-----------------+------------------+


In [ ]:
!mysql -udwid -ppw --table < scripts/scd2.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "select * from product_dim"

+------------+--------------+-----------------+------------------+----------------+-------------+
| product_sk | product_code | product_name    | product_category | effective_date | expiry_date |
+------------+--------------+-----------------+------------------+----------------+-------------+
|          1 |            1 | Hard Disk       | Storage          | 2007-02-03     | 2021-04-05  |
|          2 |            2 | Floppy Drive    | Storage          | 2007-02-03     | 9999-12-31  |
|          4 |            1 | Hard Disk Drive | Storage          | 2021-04-06     | 9999-12-31  |
|          5 |            3 | LCD Panel       | Monitor          | 2021-04-06     | 9999-12-31  |
+------------+--------------+-----------------+------------------+----------------+-------------+


In [ ]:
!mysql dw -e "update product_dim set effective_date = '2007-02-05' where effective_date = '2020-03-25'"

In [ ]:
!mysql dw -e "update product_dim set expiry_date = '2007-02-04' where expiry_date = '2020-03-24'"

In [ ]:
#there seems to be an inexplicable difference between the output shown here and the output shown in the book
#product_sk should be 1,2,3,4 and not 1,2,4,5 
#it seems autoincrements behave erratically and hence should not be used as surrogate keys as shown here

In [ ]:
!mysql dw -e "select * from product_dim"

+------------+--------------+-----------------+------------------+----------------+-------------+
| product_sk | product_code | product_name    | product_category | effective_date | expiry_date |
+------------+--------------+-----------------+------------------+----------------+-------------+
|          1 |            1 | Hard Disk       | Storage          | 2007-02-03     | 2021-04-05  |
|          2 |            2 | Floppy Drive    | Storage          | 2007-02-03     | 9999-12-31  |
|          4 |            1 | Hard Disk Drive | Storage          | 2021-04-06     | 9999-12-31  |
|          5 |            3 | LCD Panel       | Monitor          | 2021-04-06     | 9999-12-31  |
+------------+--------------+-----------------+------------------+----------------+-------------+


#Chap 3 -  Measure Additivity

In [ ]:
!cat scripts/additive_data.sql

/*********************************************************************/
/*                                                                   */
/* additive_data.sql                                                 */                  
/*                                                                   */
/*********************************************************************/

USE dw;

INSERT INTO order_dim VALUES
  (NULL, 1, CURRENT_DATE, '9999-12-31')
, (NULL, 2, CURRENT_DATE, '9999-12-31')
, (NULL, 3, CURRENT_DATE, '9999-12-31')
, (NULL, 4, CURRENT_DATE, '9999-12-31')
, (NULL, 5, CURRENT_DATE, '9999-12-31')
, (NULL, 6, CURRENT_DATE, '9999-12-31')
, (NULL, 7, CURRENT_DATE, '9999-12-31')
, (NULL, 8, CURRENT_DATE, '9999-12-31')
, (NULL, 9, CURRENT_DATE, '9999-12-31')
, (NULL, 10, CURRENT_DATE, '9999-12-31')
;

INSERT INTO date_dim VALUES
  (NULL, '2005-10-31', 'October', 10, 4, 2005, CURRENT_DATE, '9999-12-31')
;

INSERT INTO sales_order_fact VALUES
  (1, 1, 2, 1, 1000)
, (2, 2, 3, 1, 1

In [ ]:
!mysql -udwid -ppw --table < scripts/additive_data.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "select * from sales_order_fact"

+----------+-------------+------------+---------------+--------------+
| order_sk | customer_sk | product_sk | order_date_sk | order_amount |
+----------+-------------+------------+---------------+--------------+
|        1 |           1 |          2 |             1 |      1000.00 |
|        2 |           2 |          3 |             1 |      1000.00 |
|        3 |           3 |          4 |             1 |      4000.00 |
|        4 |           4 |          2 |             1 |      4000.00 |
|        5 |           5 |          3 |             1 |      6000.00 |
|        6 |           1 |          4 |             1 |      6000.00 |
|        7 |           2 |          2 |             1 |      8000.00 |
|        8 |           3 |          3 |             1 |      8000.00 |
|        9 |           4 |          4 |             1 |     10000.00 |
|       10 |           5 |          2 |             1 |     10000.00 |
+----------+-------------+------------+---------------+--------------+


In [ ]:
!mysql -udwid -ppw --table < scripts/across_all_dimensions.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
+---------------------+
| sum_of_order_amount |
+---------------------+
|            58000.00 |
+---------------------+


In [ ]:
!mysql -udwid -ppw --table < scripts/across_date_product.sql
# note this adds up to 58000 as well, as it should

mysql: [Warning] Using a password on the command line interface can be insecure.
+-----------------+---------------------+
| customer_number | sum_of_order_amount |
+-----------------+---------------------+
|               1 |             7000.00 |
|               2 |             9000.00 |
|               3 |            12000.00 |
|               4 |            14000.00 |
|               5 |            16000.00 |
+-----------------+---------------------+


In [ ]:
%%writefile pm_across_date_customer_order.sql
USE dw;
SELECT
product_code, SUM(order_amount) sum_of_order_amount
FROM
sales_order_fact a
, product_dim b
WHERE
a.product_sk = b.product_sk
GROUP BY
product_code
;
/* end of script */

Writing pm_across_date_customer_order.sql


In [ ]:
!mysql -udwid -ppw --table < pm_across_date_customer_order.sql
# this does not add up to 58,000 ! error !! 

mysql: [Warning] Using a password on the command line interface can be insecure.
+--------------+---------------------+
| product_code | sum_of_order_amount |
+--------------+---------------------+
|            1 |            20000.00 |
|            2 |            23000.00 |
+--------------+---------------------+


In [ ]:
#trying to fix the error referred to above
!mysql dw -e "update product_dim set product_sk = 3 where product_sk = 4"
!mysql dw -e "update product_dim set product_sk = 4 where product_sk = 5"


In [ ]:
%%writefile pm_quickcheck.sql
USE dw;
SELECT * from sales_order_fact;
select * from product_dim;


Writing pm_quickcheck.sql


In [ ]:
!mysql -udwid -ppw --table < pm_quickcheck.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
+----------+-------------+------------+---------------+--------------+
| order_sk | customer_sk | product_sk | order_date_sk | order_amount |
+----------+-------------+------------+---------------+--------------+
|        1 |           1 |          2 |             1 |      1000.00 |
|        2 |           2 |          3 |             1 |      1000.00 |
|        3 |           3 |          4 |             1 |      4000.00 |
|        4 |           4 |          2 |             1 |      4000.00 |
|        5 |           5 |          3 |             1 |      6000.00 |
|        6 |           1 |          4 |             1 |      6000.00 |
|        7 |           2 |          2 |             1 |      8000.00 |
|        8 |           3 |          3 |             1 |      8000.00 |
|        9 |           4 |          4 |             1 |     10000.00 |
|       10 |           5 |          2 |             1 |     10000.0

In [ ]:
!mysql -udwid -ppw --table < scripts/across_date_customer.sql
# now we get back 58,000

mysql: [Warning] Using a password on the command line interface can be insecure.
+--------------+---------------------+
| product_code | sum_of_order_amount |
+--------------+---------------------+
|            1 |            15000.00 |
|            2 |            23000.00 |
|            3 |            20000.00 |
+--------------+---------------------+


In [ ]:
#after the error fix, back in sync with the tutorial book

In [ ]:
!mysql -udwid -ppw --table < scripts/across_date.sql
# and we are back to 58,000 again

mysql: [Warning] Using a password on the command line interface can be insecure.
+-----------------+--------------+---------------------+
| customer_number | product_code | sum_of_order_amount |
+-----------------+--------------+---------------------+
|               1 |            2 |             1000.00 |
|               1 |            3 |             6000.00 |
|               2 |            1 |             1000.00 |
|               2 |            2 |             8000.00 |
|               3 |            1 |             8000.00 |
|               3 |            3 |             4000.00 |
|               4 |            2 |             4000.00 |
|               4 |            3 |            10000.00 |
|               5 |            1 |             6000.00 |
|               5 |            2 |            10000.00 |
+-----------------+--------------+---------------------+


#Chap 4 - Dimensional Queries

In [ ]:
!ls scripts/da*

scripts/daily_aggregation.sql	   scripts/daily_production.sql
scripts/daily_date.sql		   scripts/database_view.sql
scripts/daily_production_23.sql    scripts/date_view_22.sql
scripts/daily_production_data.sql  scripts/date_views.sql


In [ ]:
!cat scripts/dimensional_query_data.sql 

/*********************************************************************/
/*                                                                   */
/* dimensional_query_data.sql                                        */                  
/*                                                                   */
/*********************************************************************/

USE dw;

INSERT INTO order_dim VALUES
  (NULL, 11, CURRENT_DATE, '9999-12-31')
, (NULL, 12, CURRENT_DATE, '9999-12-31')
, (NULL, 13, CURRENT_DATE, '9999-12-31')
, (NULL, 14, CURRENT_DATE, '9999-12-31')
, (NULL, 15, CURRENT_DATE, '9999-12-31')
, (NULL, 16, CURRENT_DATE, '9999-12-31')
;

INSERT INTO date_dim VALUES
  (NULL, '2005-11-01', 'November', 11, 4, 2005, CURRENT_DATE, '9999-12-31')
;

INSERT INTO sales_order_fact VALUES
  (11, 1, 2, 2, 20000) 
, (12, 2, 3, 2, 25000)
, (13, 3, 4, 2, 30000)
, (14, 4, 2, 2, 35000) 
, (15, 5, 3, 2, 40000)
, (16, 1, 4, 2, 45000)
;

/* end of script                                

In [ ]:
!mysql dw -udwid -ppw --table < scripts/dimensional_query_data.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "select * from sales_order_fact"

+----------+-------------+------------+---------------+--------------+
| order_sk | customer_sk | product_sk | order_date_sk | order_amount |
+----------+-------------+------------+---------------+--------------+
|        1 |           1 |          2 |             1 |      1000.00 |
|        2 |           2 |          3 |             1 |      1000.00 |
|        3 |           3 |          4 |             1 |      4000.00 |
|        4 |           4 |          2 |             1 |      4000.00 |
|        5 |           5 |          3 |             1 |      6000.00 |
|        6 |           1 |          4 |             1 |      6000.00 |
|        7 |           2 |          2 |             1 |      8000.00 |
|        8 |           3 |          3 |             1 |      8000.00 |
|        9 |           4 |          4 |             1 |     10000.00 |
|       10 |           5 |          2 |             1 |     10000.00 |
|       11 |           1 |          2 |             2 |     20000.00 |
|     

In [ ]:
!cat scripts/daily_aggregation.sql

/*********************************************************************/
/*                                                                   */
/* daily_aggregation.sql                                             */                  
/*                                                                   */
/*********************************************************************/

SELECT date, SUM(order_amount), COUNT(*)
FROM
  sales_order_fact a
, date_dim b
WHERE
a.order_date_sk = b.date_sk
GROUP BY date
ORDER BY date
;

/* end of script                                                     */


In [ ]:
!mysql dw -udwid -ppw --table < scripts/daily_aggregation.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
+------------+-------------------+----------+
| date       | SUM(order_amount) | COUNT(*) |
+------------+-------------------+----------+
| 2005-10-31 |          58000.00 |       10 |
| 2005-11-01 |         195000.00 |        6 |
+------------+-------------------+----------+


In [ ]:
!mysql dw -udwid -ppw --table < scripts/annual_aggregation.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
+------+-----------------+---------------+-------------------+----------+
| year | product_name    | customer_city | SUM(order_amount) | COUNT(*) |
+------+-----------------+---------------+-------------------+----------+
| 2005 | Floppy Drive    | Mechanicsburg |          70000.00 |        5 |
| 2005 | Floppy Drive    | Pittsburgh    |           8000.00 |        1 |
| 2005 | Hard Disk Drive | Mechanicsburg |          46000.00 |        2 |
| 2005 | Hard Disk Drive | Pittsburgh    |          34000.00 |        3 |
| 2005 | LCD Panel       | Mechanicsburg |          61000.00 |        3 |
| 2005 | LCD Panel       | Pittsburgh    |          34000.00 |        2 |
+------+-----------------+---------------+-------------------+----------+


In [ ]:
!mysql dw -udwid -ppw --table < scripts/monthly_storage.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
+-----------------+------------+------+-------------------+----------+
| product_name    | month_name | year | SUM(order_amount) | COUNT(*) |
+-----------------+------------+------+-------------------+----------+
| Floppy Drive    | November   | 2005 |          55000.00 |        2 |
| Hard Disk Drive | November   | 2005 |          65000.00 |        2 |
| Floppy Drive    | October    | 2005 |          23000.00 |        4 |
| Hard Disk Drive | October    | 2005 |          15000.00 |        3 |
+-----------------+------------+------+-------------------+----------+


In [ ]:
!mysql dw -udwid -ppw --table < scripts/quarterly_mechanicsburg.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
+---------------+---------+------+-------------------+-----------------+
| customer_city | quarter | year | SUM(order_amount) | COUNT(order_sk) |
+---------------+---------+------+-------------------+-----------------+
| Mechanicsburg |       4 | 2005 |         177000.00 |              10 |
+---------------+---------+------+-------------------+-----------------+


In [ ]:
!cat scripts/monthly_product_performer.sql
# there is an error in this script .. can you spot it?

/*********************************************************************/
/*                                                                   */
/* monthly_product_performer.sql                                     */                  
/*                                                                   */
/*********************************************************************/

USE dw;

SELECT month_name, year, product_name, SUM(order_amount), COUNT(*)
FROM
  sales_order_fact a
, product_dim b
, date_dim c
WHERE
    a.product_sk = b.product_sk
AND a.order_date_sk = c.date_sk
GROUP BY 
  month_name
, year
, product_name
HAVING SUM(order_amount) >= 75000
ORDER BY 
  month
, year
, product_name
;

/* end of script                                                     */


In [ ]:
!mysql dw -udwid -ppw --table < scripts/monthly_product_performer.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1055 (42000) at line 9: Expression #1 of ORDER BY clause is not in GROUP BY clause and contains nonaggregated column 'dw.c.month' which is not functionally dependent on columns in GROUP BY clause; this is incompatible with sql_mode=only_full_group_by


In [ ]:
%%writefile pm_monthly_product_performer.sql

USE dw;

SELECT month_name, year, product_name, SUM(order_amount), COUNT(*)
FROM
  sales_order_fact a
, product_dim b
, date_dim c
WHERE
    a.product_sk = b.product_sk
AND a.order_date_sk = c.date_sk
GROUP BY 
  month_name
, year
, product_name
HAVING SUM(order_amount) >= 75000
ORDER BY 
  month_name
, year
, product_name
;

Writing pm_monthly_product_performer.sql


In [ ]:
#running the corrected script
!mysql dw -udwid -ppw --table < pm_monthly_product_performer.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
+------------+------+--------------+-------------------+----------+
| month_name | year | product_name | SUM(order_amount) | COUNT(*) |
+------------+------+--------------+-------------------+----------+
| November   | 2005 | LCD Panel    |          75000.00 |        2 |
+------------+------+--------------+-------------------+----------+


sample forms https://colab.research.google.com/notebooks/forms.ipynb

In [ ]:
!mysql dw -udwid -ppw --table < scripts/loyal_customer.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
+-----------------+------+----------+
| customer_number | year | COUNT(*) |
+-----------------+------+----------+
|               1 | 2005 |        4 |
+-----------------+------+----------+


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novel, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)